# Datapalooza: Text Mining

¡Bienvenidos a Datapalooza UC, edición Text Mining! En este taller de 4 horas, nos introduciremos en el análisis de textos con Python. Naturalmente, el Procesamiento del Lenguaje Natural y el Text Mining son dos técnicas que, en su profundidad, nos tomaría mucho más que 4 horas para explotar todo su potencial. Por esto, los módulos de este taller estarán centrados en que conozcan dos librerías para iniciarse en el Text Mining: [`spaCy`](https://spacy.io/), y su hermano menor, [`textaCy`](https://textacy.readthedocs.io/en/latest/). 

Con estas librerías, podremos realizar tareas como la obtención de frecuencias léxicas, la extracción de entidades nombradas (NER), colocaciones, etc. Además, con la ayuda de [`pandas`](https://pandas.pydata.org/), podremos lograr la tarea definitiva del Text Mining: transformar **datos no estructurados** (i.e. los textos) en **datos estructurados** (i.e. tablas) que puedan ser analizados y accionados con mayor facilidad.

Siguiendo una metodología basada en proyectos, nuestra meta al finalizar el taller será obtener conclusiones a partir de un corpus de entradas de Wikipedia. A lo largo del taller, revisaremos las entradas de tres héroes patrios: José Miguel Carrera, Bernardo O'Higgins y Manuel Rodríguez.

**Entrada de José Miguel Carrera**
> José Miguel de la Carrera y Verdugo (Santiago, 15 de octubre de 17852​3​-Mendoza, 4 de septiembre de 1821) fue un político y militar chileno, prócer de la emancipación de Chile y destacado participante en las guerras de independencia. Es reconocido como uno de los «padres de la Patria de Chile», jefe de gobierno, el primer general en jefe del Ejército y el primer caudillo en la historia republicana de dicho país, y uno de los primeros de América.4​

> Era descendiente de una familia aristocrática hispana y tras servir a las armas del rey de España en contra del ejército de Napoleón, volvió a Chile en julio de 1811. Después de sucesivos golpes de Estado, el 15 de noviembre de dicho año se hizo nombrar presidente de la Junta Provisional de Gobierno de la actualmente llamada Patria Vieja y, tras disolver el Congreso Nacional, asumió plenos poderes el 2 de diciembre. Su gobierno, abiertamente separatista con respecto al aparato estatal de España, tuvo que enfrentar la invasión que el virrey Abascal mandó a realizar desde Talcahuano, desencadenando así la Guerra por la Independencia de Chile.

¿Cuál será nuestra metodología de trabajo? Primero revisaremos **un poco de teoría detrás del PLN** y las herramientas que utilizaremos el día de hoy. Luego, **nos uniremos en equipos pequeños**, ejecutaremos en equipos análisis exploratorios sobre el corpus de trabajo y finalizaremos (si nos da el tiempo) con una pequeña presentación **exponiendo las conclusiones sobre su corpus** de trabajo.

## Primero lo primero: instalar librerías

Algunas de las librerías que utilizaremos hoy no vienen por defecto en Google Colaboratory, por lo que deberemos partir instalándolas. En este ambiente, usaremos la clásia línea de comando `pip install` antecedida por `!`. Como no nos interesa todo lo que imprimen estas instalaciones, usamos `%%capture`.

In [ ]:
%%capture 
#!python -m spacy download es_core_news_sm # modelo de tamaño pequeño para el español (12 MB)
!python -m spacy download es_core_news_md # modelo de medio tamaño para el español (40 MB)
!pip install textacy # herramientas de análisis textual basado en spacy
!pip install wikipedia # librería para recuperar fácilmente datos de Wikipedia


De los comando anteriores, particularmente importante es la línea `!python -m spacy download es_core_news_md`. Con esta línea, estamos instalando el modelo entrenado para el español de tamaño medio. Existen diferentes modelos, con diferentes tamaños y usos. Para este taller, usaremos el `es_core_news_md` por los beneficios en eficiancia de cálculo que provee. La web de [spaCy Models](https://spacy.io/models) tiene la documentación para cada uno de los modelos disponibles.

Dicho eso, cargamos las librerías que utilizaremos en este taller en nuestro ambiente de Colab.

In [ ]:
%%capture
import spacy # importa modelos y herramientas de NLP
import textacy # importa herramientas de análisis
from textacy.extract import keyterms as kt # recuperar keywords con algoritmo YAKE
from textacy import extract # extraer estructura oracional
import pandas as pd # construir tablas/DataFrame
from spacy.tokens import Span # recuperar los span para entidades nombradas
from spacy import displacy

## Cargar el modelo nlp de spaCy

Uno de los primeros pasos cuando trabajamos textos con `spaCy` es la carga del modelo de NLP. Con la carga de este modelo, en este caso de `es_core_news_md`, cargamos métodos y atributos como el Reconocimiento de Entidades Nombradas, la separación de tokens y oraciones, el vocabulario entrenado, etc. 

In [ ]:
nlp = spacy.load("es_core_news_md")

## Un primer vistazo al uso de `nlp`

Ya con nuestro modelo cargado, podemos hacer un primer uso de este. Por ejemplo, el texto sobre José Miguel Carrera y Verdugo podemos procesarlo para observar cuáles entidades están siendo nombradas, y así categorizarlas (e.g. `PERSONA`, `LOCACION`, `MISCELANEO`).

Para esto, solo almacenamos el texto crudo en una variable `TEXTO`y se la pasamos al modelo cargado `nlp(TEXTO)`:

> José Miguel de la Carrera y Verdugo (Santiago, 15 de octubre de 1785 ​- Mendoza, 4 de septiembre de 1821) fue un político y militar chileno, prócer de la emancipación de Chile y destacado participante en las guerras de independencia. Es reconocido como uno de los «padres de la Patria de Chile», jefe de gobierno, el primer general en jefe del Ejército y el primer caudillo en la historia republicana de dicho país, y uno de los primeros de América.

In [ ]:
TEXTO = "José Miguel de la Carrera y Verdugo (Santiago, 15 de octubre de 1785 ​- Mendoza, 4 de septiembre de 1821) fue un político y militar chileno, prócer de la emancipación de Chile y destacado participante en las guerras de independencia. Es reconocido como uno de los «padres de la Patria de Chile», jefe de gobierno, el primer general en jefe del Ejército y el primer caudillo en la historia republicana de dicho país, y uno de los primeros de América."
doc = nlp(TEXTO)

for ent in doc.ents:
  print(ent.text, ent.label_)

José Miguel de la Carrera y Verdugo PER
Santiago LOC
Mendoza LOC
Chile LOC
Patria de Chile MISC
Ejército MISC
América LOC


Si bien está etiquetado y sabemos que, por ejemplo, _América_ es una `LOCACION`, sigue siendo texto crudo y difícil de procesar por el computador. Ante esto, siempre podemos transformar nuestros datos en un `DataFrame` con `pandas`.

In [ ]:
carrera_verdugo = []
for ent in doc.ents:
  carrera_verdugo.append(
      {
          "texto": ent.text,
          "entidad": ent.label_
      }
  )
  
carrera_verdugo_df = pd.DataFrame(carrera_verdugo)
carrera_verdugo_df

,texto,entidad
0,José Miguel de la Carrera y Verdugo,PER
1,Santiago,LOC
2,Mendoza,LOC
3,Chile,LOC
4,Patria de Chile,MISC
5,Ejército,MISC
6,América,LOC


In [ ]:
pd.DataFrame(carrera_verdugo_df).value_counts("entidad")

entidad
LOC     4
MISC    2
PER     1
dtype: int64

Para esta breve introducción de José Miguel Carrera, vemos que se nombran 4 `LOC`, 2 `MISC` y solo 1 `PER`: _José Miguel Carrera_. No es de sorprender, considerando que es el primer párrafo de Wikipedia sobre el libertador, todo se centra en él, dónde y cuándo nació y una que otra información relevante ¿Pero qué pasa si vemos su página completa?

In [ ]:
import wikipedia
wikipedia.set_lang("es")
jmc_wiki = wikipedia.page("José Miguel Carrera")
doc = nlp(jmc_wiki.content)

carrera_verdugo = []
for ent in doc.ents:
  carrera_verdugo.append(
      {
          "texto": ent.text,
          "entidad": ent.label_
      }
  )

carrera_verdugo_df = pd.DataFrame(carrera_verdugo)
carrera_verdugo_df

,texto,entidad
0,José Miguel de la Carrera y Verdugo,PER
1,Santiago,LOC
2,Chile,LOC
3,Patria de Chile,MISC
4,Ejército,MISC
...,...,...
862,José Miguel Carrera.\n\n,PER
863,Wikiquote,MISC
864,José Miguel Carrera,PER
865,"Instituto de Investigaciones Históricas ""General",ORG


In [ ]:
carrera_verdugo_df.value_counts("entidad").head(15)

entidad
PER     405
LOC     259
MISC    138
ORG      65
dtype: int64

In [ ]:
carrera_verdugo_df.value_counts("texto").head(15)

texto
Carrera                84
Chile                  38
José Miguel            19
San Martín             17
Alvear                 15
Mendoza                13
José Miguel Carrera    13
Argentina              12
O'Higgins              10
Santiago               10
Juan José               9
Buenos Aires            9
Montevideo              9
Luis                    9
Manuel Rodríguez        8
dtype: int64

In [ ]:
def get_ngrams(doc, ngram=2, min_freq=5):
  ngrams = list(textacy.extract.basics.ngrams(doc, ngram, min_freq=min_freq))
  ngrama = []

  for ngram in ngrams:
    words = "".join(str(ngram))
    if words != "==":
      ngrama.append(
          {
              "ngram": words
          }
      )
  return ngrama

ngramas = get_ngrams(doc, ngram=5, min_freq=1)
pd.DataFrame(ngramas).value_counts("ngram")

ngram
Golpe de Estado de José                 2
Director Supremo de las Provincias      2
Supremo de las Provincias Unidas        2
Argentina ====                          2
José Miguel de la Carrera               2
                                       ..
autoridad -y posiblemente del deseo-    1
autores que dudan del republicanismo    1
autonomía dentro del Imperio español    1
ausencia del regimiento del combate     1
única solución era ese golpe            1
Length: 1184, dtype: int64

Con su página completa, vemos como la tendencia inicial cambia radicalmente. Ahora, la presencia de `PER` se hace evidente, casi duplicando `LOC`, algo esperable de un corpus histórico como este. Además, entidades `MISC` como _Guerra por la Independencia de Chile_ o _Patria Vieja_ aparecen y, si bien son reconoidas como entidades relevantes, la etiqueta es poco informativa. Una aplicación que analice y conecte personas y eventos históricos, como periodos o guerra, podría entrenar el modelo con una etiqueta nueva como `{"GUERRA": "Guerra por la Independencia de Chile}` o `{"PERIODO": "Patria Vieja"}`. 

Otra cosa que salta a la vista son los errores de etiquetado, como _Después_ etiquetado como `MISC`, o la debatible clasificación de _Congreso Nacional_ como `LOC` en lugar de `ORG`.

### Ejercicio 1: explora otro corpus de Wikipedia y observa sus entidades nombradas

1.   ¿Cómo se comportan las entidades nombradas entre páginas de sobre temas diferentes a la historia/biografías?
2.   ¿En qué contextos las entidades `MISC` podrían ser recategorizadas? ¿Qué caso de uso se les ocurre con los `MISC` de sus corpus?



In [ ]:
# Trabaja en esta celda. Si lo necesitas, puedes añadir más celdas para procesar los conteos
tu_wiki_page = wikipedia.page("_____")

doc = nlp(______)
tu_wiki_page_array = []
for ent in doc.ents:
  tu_wiki_page_array.append(
      {
          "texto": ent.text,
          "entidad": ent.______
      }
  )
  
_____ = pd.DataFrame(_____)
______.head(20)

PageError: ignored

## ¿Y si queremos entidades personalizadas?

El poder de spaCy radica en la flexibilidad que nos entrega. En términos de NLP, los modelos para el inglés funcionan increiblemente bien y logran clasificar con muy buenos resultados. Esto hace necesario que, en español, podemos entrenar las entidades que son de nuestro interés y uso particular. Una buena herramienta para crear patrones es el [Rule-based Matcher Explorer](https://demos.explosion.ai/matcher?text=Battle%20of%20Chacabuco&model=en_core_web_sm&pattern=%5B%7B%22id%22%3A0%2C%22attrs%22%3A%5B%7B%22name%22%3A%22LOWER%22%2C%22value%22%3A%22battle%22%7D%5D%7D%2C%7B%22id%22%3A1%2C%22attrs%22%3A%5B%7B%22name%22%3A%22POS%22%2C%22value%22%3A%22ADP%22%7D%5D%7D%2C%7B%22id%22%3A2%2C%22attrs%22%3A%5B%7B%22name%22%3A%22POS%22%2C%22value%22%3A%22PROPN%22%7D%5D%7D%5D). En su [documentación](https://demos.explosion.ai/matcher?text=Battle%20of%20Chacabuco&model=en_core_web_sm&pattern=%5B%7B%22id%22%3A0%2C%22attrs%22%3A%5B%7B%22name%22%3A%22LOWER%22%2C%22value%22%3A%22battle%22%7D%5D%7D%2C%7B%22id%22%3A1%2C%22attrs%22%3A%5B%7B%22name%22%3A%22POS%22%2C%22value%22%3A%22ADP%22%7D%5D%7D%2C%7B%22id%22%3A2%2C%22attrs%22%3A%5B%7B%22name%22%3A%22POS%22%2C%22value%22%3A%22PROPN%22%7D%5D%7D%5D) podrán encontrar toda la información sobre los tipos de patrones reconocibles, y también pueden apoyarse en [displaCy](https://demos.explosion.ai/displacy) para reconocer los POS (Part-Of-Speech).

En este punto, **es muy útil recordar nuestras clases de gramática del colegio y repasar las categorías gramaticales principales**. Para esto, usemos displaCy.


In [ ]:
TEXTO = "¡Hey!, esta oración tiene muchos verbos bonitos por espacio, ¿pero tiene preposiciones?"
doc = nlp(TEXTO)

displacy.render(doc, style='dep', jupyter=True, options={'distance': 90})

En la oración anterior, podemos encontrar los siguientes POS:


1.   **Puntuación** (PUNCT): signos de puntuación en general
2.   **Interjección** (INTJ): expresiones que conllevan sentimientos vivos, llamados o acciones sonoras.
3.   **Determinante** (DET): conjunto cerrado de palabras que especifican un sustantivo (NOUN) (e.g. *este, la, un*).
4.   **Verbos** (VERB): conjunto abierto de palabras que denotan una acción, estado o proceso realizado o afectado por un sustantivo (NOUN).
5.   **Adjetivo** (ADJ): conjunto abierto de palabras que modifica a los sustantivos (NOUN), entregando características o atributos a este.
6.   **Adverbio** (ADV): conjunto abierto de palabras que modifica a los adjetivos y verbos a través de circunstancias (e.g. tiempo, manera, espacio, etc).
7. **Conjunción** (CCONJ): conjunto cerrado de palabras que permiten relacionar un conjunto de elementos entre sí (e.g. *y*, *pero*, *o*, etc.)
8. **Preposición** (ADP): también conocida como Adposición, es un conjunto cerrado de palabras que permite introducir nueva información complementaria, conocida como "adjuntos" (e.g. "Salí de vacaciones **por este mes**).



Gracias a estos POS, podemos contruir patrones que nos permitan identificar y almacenar diferentes entidades personalizadas, como veremos ahora.

### Crear un patrón personalizado
**Creemos ahora una patròn que reconozca las guerras y batallas en las que estuvo asociado José Miguel Carrera.**

Por ejemplo, es común que las batallas se nombren a partir de las localidades donde ocurren, como la *Batalla de Chacabuco*. Podemos encontrar esta batalla con el siguiente patrón:

```
pattern_batalla = [{'LOWER': 'batalla'}, # reconocemos 'Batalla' en su forma minúscula
           {'POS': 'ADP'}, # Matcheamos una preposición con el POS 'ADP'
           {"POS": "DET", "OP": "?"}, # Matchemos un POS DET si está presente (le decimos que es opcional con "OP":"?")
           {'POS': 'PROPN'}] # Matcheamos un nombre propio PROPN con su POS
```

Con este patrón, podemos reconocer batallas como _batalla de Chacabuco_ (`batalla ADP ? PROPN`) o _batalla de Los Papeles_ (`batalla ADP DET PROPN`).

In [ ]:
from spacy.matcher import Matcher
from spacy.util import filter_spans

jmc_wiki = wikipedia.page("José Miguel Carrera")
doc = nlp(jmc_wiki.content)
matcher = Matcher(nlp.vocab)

# Escribamos un patrón que identifique las guerras y batallas presentes en el texto
# Por ejemplo, "Guerra del pacífico", "Batalla de Placilla"
pattern_batalla = [{'LOWER': 'batalla'},
           {'POS': 'ADP'},
           {"POS": "DET", "OP": "?"},
           {'POS': 'PROPN'}]

pattern_guerra = [{'LOWER': 'guerra'},
           {'POS': 'ADP'},
           {"POS": "DET", "OP": "?"},
           {'POS': 'PROPN'}]

# Añade el patrón al matcher y usa el matcher sobre el documento
matcher.add("BATALLA", [pattern_batalla])
matcher.add("GUERRA", [pattern_guerra])

In [ ]:
matches = matcher(doc)

for match_id, start, end in matches:
  string_id = nlp.vocab.strings[match_id] 
  span = Span(doc, start, end, label=match_id)
  doc.set_ents(entities=[span], default="unmodified")
  print("Resultado encontrado:", doc[start:end].text, nlp.vocab.strings[match_id])

Resultado encontrado: Guerra por la Independencia GUERRA
Resultado encontrado: batalla de Rancagua BATALLA
Resultado encontrado: Guerra de la Independencia GUERRA
Resultado encontrado: batalla de Talavera BATALLA
Resultado encontrado: batalla de Ocaña BATALLA
Resultado encontrado: Batalla de los Papeles BATALLA
Resultado encontrado: batalla de Chacabuco BATALLA
Resultado encontrado: Batalla de Maipú BATALLA
Resultado encontrado: batalla de la Cañada BATALLA
Resultado encontrado: Guerra del Pacífico GUERRA


Si bien en este caso creamos un patrón que distingue entre `GUERRA` y `BATALLA`, también podríamos crear un patrón con atributos (como `IN` o `NOT_IN`) para que las guerras y batallas pertenezcan al mismo tipo de entidad. 

Si vemos los patrones, veremos que ambos tienen en común la estructura `NOUN ADP DET PROPN`, distinguiéndose exclusivamente en la instancia de `NOUN` (_guerra_ vs _batalla_). Podemos, entonces, generalizar la categoría con `CONFLICTO` usando el operador `IN`, como en


```
{'LOWER': {'IN': ['batalla','guerra'] }}
```

Esto hará que el patrón busque ambas palabras para iniciar el match y las clasifique como estimemos conveniente.

In [ ]:
# Eliminemos los matches GUERRA y BATALLA antes
matcher.remove("BATALLA")
matcher.remove("GUERRA")

pattern_conflicto = [{'LOWER': {'IN':['batalla','guerra']}},
           {'POS': 'ADP'},
           {"POS": "DET", "OP": "?"},
           {'POS': 'PROPN'}]

# Añade el patrón al matcher y usa el matcher sobre el documento
matcher.add("CONFLICTO", [pattern_conflicto])

Podemos añadir estos resultados al DataFrame `carrera_verdugo_df` que creamos previamente, y así poder contabilizar las entidades presentes:

In [ ]:
matcher = Matcher(nlp.vocab)
pattern_conflicto = [{'LOWER': {'IN':['batalla','guerra']}},
           {'POS': 'ADP'},
           {"POS": "DET", "OP": "?"},
           {'POS': 'PROPN'}]

# Añade el patrón al matcher y usa el matcher sobre el documento
matcher.add("CONFLICTO", [pattern_conflicto])
matches = matcher(doc)

# Busca el patrón dado y lo añade al conjunto de entidades
for match_id, start, end in matches:
  string_id = nlp.vocab.strings[match_id] 
  span = Span(doc, start, end, label=match_id)
  doc.set_ents(entities=[span], default="unmodified") # con .set_ents configuramos las nuevas entidades para `doc`.


# Creamos un array donde iremos añadiendo las entidades "clásicas" (PER, LOC, etc.) más las personalizadas (CONFLICTO)
jmc_array = []
for ent in doc.ents:
    jmc_array.append(
        {
            "texto": ent.text,
            "entidad": ent.label_
        }
    )

# veamos como queda la tabla
carrera_verdugo_df = pd.DataFrame(jmc_array)

**¿Cuántas entidades `CONFLICTO` podemos encontrar una vez que realizamos esta clasificación?**

In [ ]:
carrera_verdugo_df.value_counts("entidad")

entidad
PER          405
LOC          258
MISC         133
ORG           65
CONFLICTO     10
dtype: int64

**¿Cuáles son las instancias de entidades más comunes en la Wikipedia sobre José Miguel Carrera**

In [ ]:
carrera_verdugo_df.value_counts("texto").head(15)

texto
Carrera                84
Chile                  38
José Miguel            19
San Martín             18
Alvear                 15
Mendoza                13
José Miguel Carrera    13
Argentina              12
O'Higgins              10
Santiago               10
Luis                    9
Juan José               9
Montevideo              9
Buenos Aires            9
Estados Unidos          8
dtype: int64

**¿Y si quisieramos saber cuáles conceptos son modificados por adjetivos?**

In [ ]:
matcher = Matcher(nlp.vocab)
pattern_evaluacion = [{'POS': 'NOUN'},
           {'POS': 'ADJ'}]

# Añade el patrón al matcher y usa el matcher sobre el documento
matcher.add("EVAL", [pattern_evaluacion])
matches = matcher(doc)

# Busca el patrón dado y lo añade al conjunto de entidades
for match_id, start, end in matches:
  string_id = nlp.vocab.strings[match_id] 
  span = Span(doc, start, end, label=match_id)
  doc.set_ents(entities=[span], default="unmodified") # con .set_ents configuramos las nuevas entidades para `doc`.


# Creamos un array donde iremos añadiendo las entidades "clásicas" (PER, LOC, etc.) más las personalizadas (CONFLICTO)
jmc_array = []
for ent in doc.ents:
  if ent.label_ == "EVAL":
      jmc_array.append(
          {
              "texto": ent.text,
              "sustantivo": str(ent[0:1]),
              "adjetivo": str(ent[1:2]), # Al subsetear `ent[1:2]` almacenamos solo el segundo token, el ADJ
              "entidad": ent.label_
          }
      )

# veamos como queda la tabla
jmc_separado = pd.DataFrame(jmc_array)
print(pd.DataFrame(jmc_array).value_counts("adjetivo").head(10))
print(" ")
print(pd.DataFrame(jmc_array).value_counts("texto").head(10))

adjetivo
política           8
militar            7
nacional           7
inglés             4
chilena            4
independentista    3
militares          3
patriota           3
personal           3
político           3
dtype: int64
 
texto
vida política              3
sectores populares         2
bandera nacional           2
documentos oficiales       2
proceso emancipador        2
diario militar             2
parte ignorantes           2
fuerzas patriotas          2
gobierno representativo    2
cabildo abierto            2
dtype: int64


In [ ]:
jmc_separado[["sustantivo", "adjetivo"]]

,sustantivo,adjetivo
0,militar,chileno
1,historia,republicana
2,familia,aristocrática
3,aparato,estatal
4,proceso,emancipador
...,...,...
243,nación,republicana
244,obras,audiovisuales
245,Contenido,relacionado
246,obras,originales


### Ejercicio 2: continúa explorando tu corpus de Wikipedia creando tres nuevos patrones

Crearemos ahora dos patrones personalizados. Revisa tu corpus y busca patrones regulares. Guíate de las herramientas [displaCy](https://demos.explosion.ai/displacy) para visualizar patrones y el [Rule-based Matcher Explorer](https://demos.explosion.ai/matcher) para ayudarte en la construcción del patrón.

In [ ]:
from spacy.matcher import Matcher
from spacy.util import filter_spans

matcher = Matcher(nlp.vocab)
pattern1 = [{"FLAG":"TOKEN"}]
pattern2 = [{"FLAG":"TOKEN"}]


# Añade el patrón al matcher y usa el matcher sobre el documento
matcher.add("NOMBRE_ENTIDAD", [pattern1])

tu_wiki_page_array = []
for match_id, start, end in matches:
  string_id = nlp.vocab.strings[match_id] 
  span = Span(doc, start, end, label=match_id)
  doc.set_ents(entities=[span], default="missing")
  
  # Construye el DataFrame 
  for ent in doc.ents:
    tu_wiki_page_array.append(
        {
            "texto": ent.text,
            "entidad": ent.label_
        }
    )

pd.DataFrame(tu_wiki_page_array)

## Visualizar diferencias léxicas entre dos categorías

Hasta este punto, solo hemos revisado un único texto a la vez, es decir, la entrada de Wikipedia sobre José Miguel Carrera. Con esta entrada, pudimos extraer las personas involucradas, los lugares mencionados e incluso creamos entidades nuevas, como `GUERRA` o `CONFLICTO`. 

Lo interesante de estas pequeñas pero poderosas piezas es que ahora podemos combinarlas para hacer comparaciones. Por ejemplo, podemos revisar los tipos de entidades más frecuentes entre categorías, o comparar las frecuencias de palabras entre categorías, entre otras muchas tareas, todo a través de DataFrames.


Veamos un ejemplo rápido. La celda de código siguiente llama al `Matcher`, añade los patrones de `GUERRA` y `BATALLA` al `Matcher`, setea la búsqueda de tres padres de la patria ("Bernardo O'Higgins", "José Miguel Carrera", "Manuel Rodríguez Erdoíza") en Wikipedia y almacena los resultados encontrados en un DataFrame de `pandas`.

In [ ]:
matcher = Matcher(nlp.vocab)

# Escribamos un patrón que identifique las guerras y batallas presentes en el texto
# Por ejemplo, "Guerra del pacífico", "Batalla de Placilla"
pattern_batalla = [{'LOWER': 'batalla'},
           {'POS': 'ADP'},
           {"POS": "DET", "OP": "?"},
           {'POS': 'PROPN'}]

pattern_guerra = [{'LOWER': 'guerra'},
           {'POS': 'ADP'},
           {"POS": "DET", "OP": "?"},
           {'POS': 'PROPN'}]

# Añade el patrón al matcher y usa el matcher sobre el documento
matcher.add("BATALLA", [pattern_batalla])
matcher.add("GUERRA", [pattern_guerra])


# Establece las páginas de wikipedia a buscar
padres_de_la_patria = ["Bernardo O'Higgins", "José Miguel Carrera", "Manuel Rodríguez Erdoíza"]

# Crea una lista vacía para construir el DataFrame
padres_array = []
for persona in padres_de_la_patria:
  print(persona)

  # Realiza la llamada a wikipedia
  wiki = wikipedia.page(persona)
  doc = nlp(wiki.content)
  matches = matcher(doc)

  # Establece los matches personalizados de BATALLA y GUERRA y los añade a los documentos
  for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id] 
    span = Span(doc, start, end, label=match_id)
    doc.set_ents(entities=[span], default="unmodified")
 
  # Recorre las entidades de cada documento y almacena sus entidades en la lista
  for ent in doc.ents:
    padres_array.append(
        {
            "persona": persona,
            "texto": ent.text,
            "entidad": ent.label_
        }
    )

# Crea el DataFrame, agrupa los datos por persona y entidad y devuelve un conteo agrupado
df_padres = pd.DataFrame(padres_array)
df_padres.groupby(["persona", "entidad"]).count()

Bernardo O'Higgins
José Miguel Carrera
Manuel Rodríguez Erdoíza


texto
persona                  entidad       
Bernardo O'Higgins       BATALLA      7
                         GUERRA       5
                         LOC        597
                         MISC       260
                         ORG        104
                         PER        660
José Miguel Carrera      BATALLA      7
                         GUERRA       3
                         LOC        258
                         MISC       133
                         ORG         65
                         PER        405
Manuel Rodríguez Erdoíza BATALLA      7
                         GUERRA       2
                         LOC        189
                         MISC       152
                         ORG         46
                         PER        494

Comparemos las instancias de entidades entre personas:

In [ ]:
print("ENTRADA: OHIGGINS")
print(df_padres[df_padres["persona"] == "Bernardo O'Higgins"].value_counts("texto").head(10), "\n")
print("ENTRADA: RODRIGUEZ")
print(df_padres[df_padres["persona"] == "Manuel Rodríguez Erdoíza"].value_counts("texto").head(10), "\n")
print("ENTRADA: CARRERA")
print(df_padres[df_padres["persona"] == "José Miguel Carrera"].value_counts("texto").head(10))

### Ejercicio 3: compara los verbos (VERB), adverbios (ADV) y adjetivos (ADJ) utilizados en un conjunto de textos de tu elección

Comúnente, los **verbos**, cuyo POS es VERB, suelen referir a acciones, procesos o estados que afectan o realizan los sustantivos (p.ej. *el gato comió*). Esta clase de palabras es central en el lenguaje humano, en tanto nos indica qué está pasando en un texto. 

De igual forma, los **adverbios**, cuyo POS es ADV, tienen una función similar a los adjetivos: modificar a un verbo (p.ej. *el gato comió rápidamente*). En este sentido, son claves para comprender **cómo ocurren los procesos o acciones del texto**. 

Imaginemos un conjunto de comentarios sobre una película. La audiencia podría decirnos que "la película **avanza muy lento**", "Timothee Chalamet **actuó increíblemente**" o que "el final **era claramente** predecible". 

Con todo lo que hemos visto hasta ahora, **crea patrones que te permitan comparar tres textos diferentes, guárdandolos en un DataFrame que te permita ver el conteo de palabras entre categorías**. Apóyate del código que ya has escrito anteriormente.

In [ ]:
# Escribe tu código acá

# Llama al matcher

# Crea los patrones

# añade los patrones al matcher

# Indica las páginas de wikipedia que buscaras

# Itera sobre las páginas y cárgalas en un nlp

# Itera sobre los matches y añádelos al vocabulario del doc

# Crea y pobla el DataFrame con la página, el texto y la entidad

# Visualiza el la frecuencia de cada categoría y página



## Algunos análisis finales

Por medio del hermano menor de `spaCy`, `textaCy`, podemos realizar algunos análisis fácil y rápido. 

### Extracción de keywords con YAKE

Por ejemplo, la celda siguiente ejecuta una extracción de palabras claves con el [algoritmo YAKE](https://github.com/LIAAD/yake). Con este algoritmo **no supervisado**,**independiente de corpus, dominio y lenguaje** y **orientado a documentos individuales**, nos ayudará a extraer rápidamente las palabras claves a partir de la extracción de rasgos.

In [ ]:
TEXTO = "José Miguel de la Carrera y Verdugo (Santiago, 15 de octubre de 1785 ​- Mendoza, 4 de septiembre de 1821) fue un político y militar chileno, prócer de la emancipación de Chile y destacado participante en las guerras de independencia. Es reconocido como uno de los «padres de la Patria de Chile», jefe de gobierno, el primer general en jefe del Ejército y el primer caudillo en la historia republicana de dicho país, y uno de los primeros de América."
doc = textacy.make_spacy_doc(TEXTO, lang="es_core_news_md")
print("KEYWORD", "\t", "YAKE SCORE")
list(kt.yake(doc, normalize="lemma"))

A partir de la celda anterior, notamos que, a menor el Yake Score, más relevante es la palabra en el documento (en este caso, *José Miguel*).

Dado que este algoritmo nos devuelve una lista de tuplas, donde la posición `0` es la keyword y la `1` el Yake Score, podemos iterar sobre esta y almacenarla en una DataFrame mucho más manejable:

In [ ]:
# Establece las páginas de wikipedia a buscar
padres_de_la_patria = ["Bernardo O'Higgins", "José Miguel Carrera", "Manuel Rodríguez Erdoíza"]

# Crea una lista vacía para construir el DataFrame
padres_yake_array = []
for persona in padres_de_la_patria:
  print(persona)

  # Realiza la llamada a wikipedia
  wiki = wikipedia.page(persona)
  doc = textacy.make_spacy_doc(wiki.content, lang="es_core_news_md")
  
  yake_list = list(kt.yake(doc, normalize="lemma"))

  # Itera sobre la yake_list y separa la tupla entre keyword y yake para cada documento `persona`
  for palabra in yake_list:
    padres_yake_array.append(
        {
          "persona": persona,
          "keyword": palabra[0],
          "yake": palabra[1]
        }
    )

pd.DataFrame(padres_yake_array)

### Extracción de la estructura oracional Sujeto-Verbo-Predicado

Para otros casos de uso, la extracción de la estructura oracional entre **sujeto, verbo y predicado (SVP)** puede ser útil. Con la función `extract.triples.subject_verb_object_triples` de `textaCy` podemos lograr esto:

In [ ]:

# Establece las páginas de wikipedia a buscar
padres_de_la_patria = ["Bernardo O'Higgins", "José Miguel Carrera", "Manuel Rodríguez Erdoíza"]

# Crea una lista vacía para construir el DataFrame
padres_SVP_array = []
for persona in padres_de_la_patria:
  print(persona)

  # Realiza la llamada a wikipedia
  wiki = wikipedia.page(persona)
  doc = textacy.make_spacy_doc(wiki.content, lang="es_core_news_md")
  
  SVP_list = list(extract.triples.subject_verb_object_triples(doc))

  # Si tiene dos o más palabras el verbo, será compuesto. Caso contrario, es un verbo simple
  for palabra in SVP_list:
    if len(palabra[1]) >= 2:
      tipo_verbo = "compuesto"
    else:
      tipo_verbo = "simple"
    
    # Guarda los resultados en el DataFrame padres_SVP
    padres_SVP_array.append(
        {
          "persona": persona,
          "sujeto": str(palabra[0]),
          "verbo": str(palabra[1]),
          "predicado": str(palabra[2]),
          "tipo_verbo": tipo_verbo
        }
    )

padres_SVP = pd.DataFrame(padres_SVP_array)

Con este código podemos, por ejemplo, visualizar verticalmente las oraciones con verbos compuestos (e.g. *habría decidido*) vs las oraciones con verbos simples (e.g. *decidió*). Esto nos ayuda a mirar, al menos inicialmente, que tanto Bernardo O'Higgins como Carrera son representados como agentes en sus entradas de Wikipedia (es decir, son ellos quienes realizan las acciones y modifican los eventos del predicado), mientras que Rodríguez tiene casos donde es mostrado como pasivo, sufriendo los efectos de otros sobre él (e.g. *otros acusaron a Manuel y Carlos*).

In [ ]:
padres_SVP[padres_SVP["tipo_verbo"] == "compuesto"]

Por lo general, la estructura oracional nos puede ayudar a visualizar rápidamente **quién realiza una acción**, **cuál acción se ejecuta** y **sobre qué se realiza la acción**. Además, por lo general, los elementos en la posición `SUJETO` suelen ser los temas centrales sobre los que se hablan en el texto, dando más información sobre estos. 

Esto, ciertamente, nos ayuda a dar una visualización por arriba o clasificar los tipos de verbos, pero también podríamos atacar el problema de representación agente-recipiente con un patrón que identifique las oraciones activa (*el transporte afectó la calidad del produto*) y pasivas (*la calidad del producto fue afectada por el transporte*). 

## Ejercicio final: todo a la parrilla

Hasta este punto, hemos revisado varias herramientas útiles para la transformación de datos no estructurados en datos estructurados. Hemos construido DataFrames con `pandas`, creado y extraído entidades nuevas con `spaCy`, obtenido las frecuencias de entidades y palabras con `spaCy` y `pandas` y revisado las keywords y la estructura oracional con `textacy`. **Ahora es momento de poner todo en la juguera**.


Con todas las herramientas que has visto hasta el momento (incluyendo todo lo que ya sepas desde antes), **transforma un conjunto de datos de Wikipedia en una DataFrame y realiza un análisis a partir de estos datos**. 

Intenta ocupar todas las herramientas vistas hasta el momento. Además, si conoces métodos de visualización de datos o métodos estadísticos, eres libre de ocuparlos para que te entreguen aún más información sobre el texto. 

Cuando estén listos con sus análisis, súbanlos al siguiente tablero de Miro: https://miro.com/app/board/uXjVPt7Gpfk=/?share_link_id=32129012151